In [5]:
import pprint

import openmm as mm
import openmm.app as app
from openmm import unit
import pdbfixer

## PDBFixer

In [2]:
fixer = pdbfixer.PDBFixer("data/3poz.pdb")
print(fixer.topology)
print(fixer.positions[0:3])

<Topology; 2 chains, 423 residues, 2536 atoms, 2463 bonds>
[Vec3(x=-0.032400000000000005, y=3.2759, z=-0.39270000000000005), Vec3(x=-0.0237, y=3.1343, z=-0.3461), Vec3(x=0.119, y=3.0783000000000005, z=-0.3514)] nm


In [3]:
# 分子力場の設定
forcefield = app.ForceField("amber14-all.xml","amber14/tip3pfb.xml") # 主な力場と水モデルを指定

In [4]:
# remove heterogen
fixer.removeHeterogens(keepWater=True)

In [9]:
# 残基の確認
pprint.pprint(fixer.findMissingResidues())
# 末端残基のみ削除する
chains = list(fixer.topology.chains())
keys = fixer.missingResidues.keys()
for key in list(keys):
    chain = chains[key[0]]
    if key[1] == 0 or key[1] == len(list(chain.residues())):
        del fixer.missingResidues[key]

In [12]:
# 非標準残基の置換 (3pozにはない)
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()

In [16]:
# 欠損原子の確認
fixer.findMissingAtoms()
fixer.addMissingAtoms()

In [17]:
# 水素原子の付与
ph = 7.0
fixer.addMissingHydrogens(ph)

In [18]:
# 水ボックスの追加
modeller = app.Modeller(fixer.topology,fixer.positions)
modeller.addSolvent(forcefield,padding=1.0*unit.nanometers,ionicStrength=0.15*unit.molar) # padding: 水の層の厚み、ionicStrength: NaCl濃度(positiveIon,negativeIonで変更可能)

In [19]:
# 出力
top = modeller.getTopology()
pos = modeller.getPositions()
app.PDBFile.writeFile(top,pos,open("data/3poz_processed.pdb","w"))